#  Stink Bug Pipline

This notebook grab data on the website. The data we used are city boundary, land cover, and stink bug data in Minnesota. The pipline includes data qaqc, simulation, model assessment, and pority ranking. The simulation includes three models: Monte Carlo, Gravity, and Huff model. And the priority ranking is based on the false positive predition for the current month and potential spread paths for the next a few months. The results uploaded to ArcOnline are four simulations maps (4 different months) and one priority map.

In [11]:
import arcpy
import json
import zipfile
import pprint
import requests
from arcpy import env
import os
from zipfile import ZipFile
import random
import time
import datetime
import pandas as pd
from dbfread import DBF
import numpy as np
from dateutil import relativedelta

In [48]:
# Function to download a file from a URL and unzip it
def get_data(url, local_directory):
    response = requests.get(url)
    file_name = os.path.join(local_directory, url.split("/")[-1])

    with open(file_name, "wb") as f:
        f.write(response.content)
        
    with ZipFile(file_name, "r") as zip_ref:
        zip_ref.extractall(local_directory)

# Define local directory
local_directory = r"E:\coursework\ARCGIS2\BMSMProject\data"
# Create the local directory if it doesn't exist
if not os.path.exists(local_directory):
    os.makedirs(local_directory)

# Define the URL
boundary_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/bdry_mn_city_township_unorg/shp_bdry_mn_city_township_unorg.zip"
bmsb_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_mda/biota_bmsb/shp_biota_bmsb.zip"
landcover_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_landcover_nlcd_mn_2019/tif_biota_landcover_nlcd_mn_2019.zip"

get_data(boundary_url,local_directory)
get_data(bmsb_url,local_directory)
get_data(landcover_url,local_directory)

print('Data extraction is done!')

Data extraction is done!


## Specify time interval ("REAL-TIME" DATA :))

In [64]:
#PLEASE SECLECT A TIME INTERVAL YOU WANT TO SIMULATE 
#MUST BE AS THE FORMAT XXXX-XX
#MUST WITHIN 2021-01 TO 2021-12
Sim_time = '2021-06'

Sim_time = datetime.datetime.strptime(Sim_time, '%Y-%m')
Sim_year = Sim_time.year
Sim_month = Sim_time.month

## QAQC 


### BMSB data QAQC

In [65]:
# Convert BMSB table to pandas df
bmsb_dbf = DBF(os.path.join(local_directory,'BMSBSurveyDataTable.dbf'))

bmsb_df = pd.DataFrame(iter(bmsb_dbf))

#clean and format data
bmsb_df = bmsb_df[["SiteName", "City", "Latitude", "Longitude", "Year",'CheckDate','Adults','Nymphs']].copy()
bmsb_df = bmsb_df.dropna(subset=["Latitude", "Longitude", 'CheckDate','Adults','Nymphs'])
bmsb_df["CheckDate"] = bmsb_df["CheckDate"].astype('datetime64[ns]')
bmsb_df["month"] = pd.DatetimeIndex(bmsb_df["CheckDate"]).month

#bounding box
bmsb_df = bmsb_df[(bmsb_df["Longitude"] > -97.239) & (bmsb_df["Longitude"] < -89.492) &
                              (bmsb_df["Latitude"] > 43.499) & (bmsb_df["Latitude"] < 49.384)]

#calculate bmsb_dfnum of obs
bmsb_df['ObsNum'] = bmsb_df['Adults'] + bmsb_df['Nymphs']


bmsb_df = bmsb_df[bmsb_df['ObsNum']>0]

#get the bmsb data within the time range
bmsb_df = bmsb_df[(bmsb_df['Year']<Sim_year)|((bmsb_df['Year']==Sim_year) & (bmsb_df['month']<=Sim_month))]

bmsb_df

,SiteName,City,Latitude,Longitude,Year,CheckDate,Adults,Nymphs,month,ObsNum
55,AFTON APPLE ORCHARD,HASTINGS,44.818630,-92.813420,2015.0,2015-09-23,1.0,0.0,9,1.0
122,AFTON APPLE ORCHARD,HASTINGS,44.818870,-92.813330,2018.0,2018-09-27,1.0,0.0,9,1.0
133,AFTON APPLE ORCHARD,HASTINGS,44.818654,-92.813416,2019.0,2019-09-25,3.0,0.0,9,3.0
197,ALEXIS BAILLY VINEYARD,HASTINGS,44.686010,-92.871800,2020.0,2020-09-10,1.0,0.0,9,1.0
548,BATTLE CREEK,SAINT PAUL,44.916890,-93.010860,2016.0,2016-10-12,58.0,3.0,10,61.0
...,...,...,...,...,...,...,...,...,...,...
7363,Alexis BAILLY VINEYARD,HASTINGS,44.686019,-92.870200,2021.0,2021-06-15,1.0,0.0,6,1.0
7441,AAMODT'S APPLE FARM,STILLWATER,45.043163,-92.865885,2021.0,2021-06-08,1.0,0.0,6,1.0
7468,BLAINE COMMUNITY GARDEN,BLAINE,45.164482,-93.209164,2021.0,2021-06-09,1.0,0.0,6,1.0
7485,EIDEM ESTATE COMMUNITY GARDEN,BROOKLYN PARK,45.135605,-93.336618,2021.0,2021-06-09,1.0,0.0,6,1.0


In [66]:
arcpy.env.workspace = "E:\coursework\ARCGIS2\BMSMProject\data"

sr = arcpy.SpatialReference(4326) # WGS 1984

# create a new feature class
arcpy.CreateFeatureclass_management(arcpy.env.workspace, "bmsb", "POINT",spatial_reference=sr)

# add fields to the feature class
arcpy.AddField_management("bmsb.shp", "SiteName", "Text")
arcpy.AddField_management("bmsb.shp", "City", "Text")
arcpy.AddField_management("bmsb.shp", "Latitude", "Double")
arcpy.AddField_management("bmsb.shp", "Longitude", 'Double')
arcpy.AddField_management("bmsb.shp", "CheckDate", 'Date')
arcpy.AddField_management("bmsb.shp", "month", "Short")
arcpy.AddField_management("bmsb.shp", "Year", "Short")
arcpy.AddField_management("bmsb.shp", "ObsNum", "Short")
#arcpy.AddField_management("bmsb.shp", "Presence", "Short")

# insert data into the feature class
cursor = arcpy.da.InsertCursor("bmsb.shp", ["SHAPE@", "SiteName", "City","Latitude",'Longitude','CheckDate','month','Year','ObsNum'])
for index,row in bmsb_df.iterrows():
    point = arcpy.Point(row[3], row[2])
    cursor.insertRow([point, row[0], row[1],row[2], row[3],row[5], row[8],row[4], row[9]])
del cursor

### City data QAQC

In [85]:
#saves city polygons
#expression = 
arcpy.conversion.FeatureClassToFeatureClass("city_township_unorg", 
                                            "E:\coursework\ARCGIS2\BMSMProject\data", 
                                            "City.shp", "CTU_CLASS = 'CITY'")

#make sure the spatial reference system is 4326
sr = arcpy.SpatialReference(4326) # WGS 1984
arcpy.management.Project("City.shp", r"City_reproject.shp", sr)

#merge cities with the same name
arcpy.management.Dissolve("City_reproject", "City_qaqc_done", "FEATURE_NA", [["POPULATION", 'SUM']])

# Alter name FEATURE_NA to CITY_NAME
arcpy.management.CalculateField("City_qaqc_done", "CITY_NAME", "!FEATURE_NA!")
arcpy.management.CalculateField("City_qaqc_done", "POPULATION", "!SUM_POPULA!",field_type = 'LONG')

#delete unnecessary columns
arcpy.management.DeleteField("City_qaqc_done", "CITY_NAME;POPULATION", "KEEP_FIELDS")

#convert each City to points (has been poylgons until now)
arcpy.management.FeatureToPoint("City_qaqc_done", "City_Point.shp", "CENTROID")
arcpy.management.DeleteField("City_Point", "CITY_NAME;POPULATION", "KEEP_FIELDS")

#simulation city shp layer
arcpy.CopyFeatures_management("City_Point", "City_Pred_0")
#prediction city shp layer
arcpy.CopyFeatures_management("City_Point", "City_Pred_1")
arcpy.CopyFeatures_management("City_Point", "City_Pred_2")
arcpy.CopyFeatures_management("City_Point", "City_Pred_3")

<Result 'E:\\coursework\\ARCGIS2\\BMSMProject\\data\\City_Point.shp'>

### Land Cover QAQC

In [4]:
#bounding box
arcpy.management.Clip('NLCD_2019_Land_Cover.tif', "132660 4774410 791819 5491608", 'LC.tif')

# clip landcover data to each city polygon
out_raster = arcpy.sa.ExtractByMask("LC.tif", "City_qaqc_done", "INSIDE")
out_raster.save(r"LC_City")

# Convert raster to points with the pixel count value for each point and land class
arcpy.conversion.RasterToPoint("LC_City", r"LC_City_point", "NLCD_LAND")

# Spatial Join the points to city polygon shapefile
arcpy.analysis.SpatialJoin("LC_City_point", "City_qaqc_done", r"LC_City_point_spatialjoin", "JOIN_ONE_TO_ONE", "KEEP_ALL")

# calculate the total count of pixels by city
arcpy.Statistics_analysis("LC_City_point_spatialjoin", "output_table",[["NLCD_LAND", "COUNT"]], "CITY_NAME")

attributes_to_match = [
    "Open Water",
    "Barren Land (Rock/Sand/Clay)",
    "Deciduous Forest",
    "Evergreen forest",
    "Mixed forest",
    "shrub/scrub",
    "Grassland/Herbaceous",
    "Pasture/Hay",
    "Cultivated Crops",
    "Woody Wetlands",
    "Emergent Herbaceous Wetlands"
]


# Make a feature layer from the input feature class
arcpy.MakeFeatureLayer_management("LC_City_point_spatialjoin", "LC_City_AG")

# Use a SQL expression to filter the layer based on the desired attributes
sql_expression = f"{arcpy.AddFieldDelimiters("LC_City_AG", 'NLCD_LAND')} IN {tuple(attributes_to_match)}"
arcpy.SelectLayerByAttribute_management("LC_City_AG", "NEW_SELECTION", sql_expression)

arcpy.Statistics_analysis("LC_City_AG", "output_table_agriculture", [["NLCD_LAND", "COUNT"]], "CITY_NAME")

#join two output tables
arcpy.management.AddJoin("output_table_agriculture", "CITY_NAME", "output_table",  "CITY_NAME", "KEEP_ALL")

#add field for ag_percentage
arcpy.management.AddField("output_table_agriculture", "ag_percentage", "DOUBLE")

# calculate the ag_percentage by dividng the agricultural field by total pixel count
arcpy.management.CalculateField("output_table_agriculture", "ag_percentage", "!FREQUENCY! / !FREQUENCY_1!", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")


<Result 'E:\\coursework\\ARCGIS2\\BMSMProject\\data\\LC.tif'>

### Generate City FROM-TO table

In [80]:
#generate from-to table
arcpy.analysis.GenerateNearTable("City_Point", "City_Point", "E:\coursework\ARCGIS2\BMSMProject\Default.gdb\City_FromToTable", closest = "ALL", method = "GEODESIC")

<Result 'E:\\coursework\\ARCGIS2\\BMSMProject\\Default.gdb\\City_FromToTable'>

In [81]:
arcpy.env.workspace = "E:\coursework\ARCGIS2\BMSMProject\Default.gdb"

#delete unnecessary columns
arcpy.management.DeleteField("City_FromToTable", "IN_FID;NEAR_FID;NEAR_DIST", "KEEP_FIELDS")

#format from-to city and population
arcpy.management.JoinField("City_FromToTable", "IN_FID", "City_qaqc_done", "FID", ["CITY_NAME",'POPULATION'])
arcpy.management.AlterField("City_FromToTable", "CITY_NAME", 'FROM_CITY','FROM_CITY')
arcpy.management.AlterField("City_FromToTable", "POPULATION", 'FROM_POP','FROM_POP')
arcpy.management.JoinField("City_FromToTable", "NEAR_FID", "City_qaqc_done", "FID", ["CITY_NAME",'POPULATION'])
arcpy.management.AlterField("City_FromToTable", "CITY_NAME", 'TO_CITY','TO_CITY')
arcpy.management.AlterField("City_FromToTable", "POPULATION", 'TO_POP','TO_POP')

<Result 'City_FromToTable'>

In [15]:
arcpy.JoinField_management("City_FromToTable", "TO_CITY", "output_table_agriculture", "CITY_NAME")
arcpy.TableToTable_conversion("City_FromToTable", r"E:\coursework\ARCGIS2\BMSMProject\Default.gdb", "City_FromToTable_Final")

<Result 'E:\\coursework\\ARCGIS2\\BMSMProject\\Default.gdb\\City_FromToTable_Final'>

### Monte Carlo Simulation

In [93]:
def sim_pred(FromToTable,SimLayer,method,PresenceCity,num_month,G,i):
    for j in range(0, num_month):
        with arcpy.da.SearchCursor(FromToTable,"*") as cursor:
            for row in cursor:
                if method == 'MC':
                    if row[4] in PresenceCity and row[6] not in PresenceCity and row[3]<=distance_threhold:
                        prob = max(0, random.normalvariate(0.024, 0.1 * 0.024))
                        if random.random()<= prob:
                            PresenceCity.append(row[6])
                elif method == 'G':
                    if row[4] in PresenceCity and row[6] not in PresenceCity:
                        if random.random()<= G * row[10]:
                            PresenceCity.append(row[6])
                elif method == 'H':
                    if row[4] in PresenceCity and row[6] not in PresenceCity:
                        if random.random()<= G * row[11]:
                            PresenceCity.append(row[6])
                else:
                    print('Please select methods among MC/G/H')
                    break

    if num_month > 1:
        print(f"Simulation{i}:{PresenceCity}")
        
    # adds presence column for each simulation
    arcpy.management.AddField(SimLayer, f"Pred{i}", "SHORT")
        
    with arcpy.da.UpdateCursor(SimLayer,"*") as cursor:
        for row in cursor:
            if method == 'MC':
                if row[2] in PresenceCity:
                    row[(4+i)] = 1
                else:
                    row[(4+i)] = 0
                cursor.updateRow(row)
            elif method == 'G':
                if row[2] in PresenceCity:
                    row[(5+i)] = 1
                else:
                    row[(5+i)] = 0
                cursor.updateRow(row)
            elif method == 'H':
                if row[2] in PresenceCity:
                    row[(6+i)] = 1
                else:
                    row[(6+i)] = 0
                cursor.updateRow(row)
    return PresenceCity

In [69]:
def final_result(Layer,method):
    with arcpy.da.UpdateCursor(Layer,"*") as cursor:
        for row in cursor:
            if method == 'MC':
                if sum(row[4:34])>15:
                    row[34] = 1
                else:
                    row[34] = 0
                cursor.updateRow(row)
            elif method == 'G':
                if sum(row[5:35])>15:
                    row[35] = 1
                else:
                    row[35] = 0
                cursor.updateRow(row)
            elif method == 'H':
                if sum(row[6:36])>15:
                    row[36] = 1
                else:
                    row[36] = 0
                cursor.updateRow(row)

In [94]:
# Read values from field
distance_values = []
with arcpy.da.SearchCursor("City_FromToTable_Final", ["NEAR_DIST"]) as cursor:
    for row in cursor:
        if row[0] is not None:
            distance_values.append(row[0])

# Calculate the 1st quantile
distance_threhold = np.quantile(distance_values, 0.005)

#get the number of timestep
Initial_time = datetime.datetime.strptime('2020-06', '%Y-%m')   #simulation start month 2020-06
delta = relativedelta.relativedelta(Sim_time, Initial_time)
num_month = delta.months + (delta.years * 12)


for i in range(0,30):
    PresenceCity = sim_pred("City_FromToTable_Final","City_Pred_0",method = 'MC',
                              PresenceCity = ['Minneapolis'],num_month = num_month,G = None,i = i)
    
    PresenceCity = sim_pred("City_FromToTable_Final","City_Pred_1",method = 'MC',PresenceCity = PresenceCity, num_month = 1, G = None,i = i)
    PresenceCity = sim_pred("City_FromToTable_Final","City_Pred_2",method = 'MC',PresenceCity = PresenceCity, num_month = 1, G = None,i = i)
    PresenceCity = sim_pred("City_FromToTable_Final","City_Pred_3",method = 'MC',PresenceCity = PresenceCity, num_month = 1, G = None,i = i)

Simulation0:['Minneapolis', 'Roseville', 'Fridley', 'Brooklyn Center', 'Vadnais Heights', 'West Saint Paul', 'Gem Lake', 'Birchwood Village', 'New Hope', 'Mounds View', 'Robbinsdale', 'North Oaks', 'Circle Pines', 'Medicine Lake', 'Crystal', 'Maple Grove', 'Arden Hills', 'Minnetonka', 'Shoreview', 'Wayzata', 'Woodland', 'Brooklyn Park', 'New Brighton', 'Lauderdale', 'Hilltop']
Simulation1:['Minneapolis', 'Lauderdale', 'Fridley', 'Mendota', 'Osseo', 'Shoreview', 'South Saint Paul', 'Falcon Heights', 'Richfield', 'Edina', 'Golden Valley', 'Medicine Lake', 'Bloomington', 'Brooklyn Center', 'North Saint Paul', 'Saint Louis Park', 'Hopkins', 'Mounds View', 'Eagan', 'Maplewood', 'Brooklyn Park', 'New Hope', 'Pine Springs', 'West Saint Paul', 'Plymouth', 'Crystal', 'Mendota Heights', 'Vadnais Heights', 'Coon Rapids', 'Champlin', 'Robbinsdale', 'Columbia Heights', 'Roseville', 'Lexington', 'Saint Anthony', 'Gem Lake', 'Mahtomedi', 'Oakdale', 'Saint Paul', 'Dayton', 'Lilydale', 'Newport', 'Wayz

Simulation26:['Minneapolis', 'Lilydale', 'Mendota Heights', 'Lauderdale', 'Crystal', 'Robbinsdale', 'Spring Lake Park', 'Columbia Heights', 'Eagan', 'Circle Pines', 'Osseo', 'Coon Rapids', 'Brooklyn Park', 'Richfield', 'Saint Paul', 'Fridley', 'West Saint Paul', 'North Oaks', 'New Hope', 'Lino Lakes', 'Mounds View', 'Falcon Heights', 'Hilltop', 'Plymouth', 'Mendota', 'Dayton', 'Anoka', 'Shoreview', 'Saint Anthony', 'Andover', 'Newport', 'Dellwood', 'Edina', 'Birchwood Village', 'Medicine Lake', 'Oak Grove', 'Willernie', 'Oak Park Heights', 'Blaine', 'Golden Valley', 'Rosemount', 'Vadnais Heights', 'Roseville', 'Bayport', 'Lake Elmo', 'Lakeland Shores', 'Burnsville', 'Arden Hills']
Simulation27:['Minneapolis', 'Lilydale', 'Hopkins', 'Golden Valley', 'Robbinsdale', 'Inver Grove Heights', 'Newport', 'Roseville', 'Lauderdale', 'New Hope', 'Medicine Lake', 'Cottage Grove', 'Brooklyn Park', 'West Saint Paul', 'Eden Prairie', 'Hilltop', 'Plymouth', 'Osseo', 'Afton', 'Circle Pines', 'Little Ca

In [95]:
#get the simulation final prediction
arcpy.management.AddField("City_Pred_0", f"Pred_MC", "SHORT")
arcpy.management.AddField("City_Pred_1", f"Pred_MC", "SHORT")
arcpy.management.AddField("City_Pred_2", f"Pred_MC", "SHORT")
arcpy.management.AddField("City_Pred_3", f"Pred_MC", "SHORT")

final_result("City_Pred_0",method = 'MC')        
#delete unnecessary columns
arcpy.management.DeleteField("City_Pred_0", "CITY_NAME;POPULATION;Pred_MC", "KEEP_FIELDS")

final_result("City_Pred_1",method = 'MC')
#delete unnecessary columns
arcpy.management.DeleteField("City_Pred_1", "CITY_NAME;POPULATION;Pred_MC", "KEEP_FIELDS")

final_result("City_Pred_2",method = 'MC')
#delete unnecessary columns
arcpy.management.DeleteField("City_Pred_2", "CITY_NAME;POPULATION;Pred_MC", "KEEP_FIELDS")

final_result("City_Pred_3",method = 'MC')
#delete unnecessary columns
arcpy.management.DeleteField("City_Pred_3", "CITY_NAME;POPULATION;Pred_MC", "KEEP_FIELDS")

<Result 'City_Pred_3'>

### Gravity Model - Pop & Dist

In [40]:
# adds probability column to near table
arcpy.management.AddField("City_FromToTable_Final", "Probability_G", "DOUBLE")

#Calculates probability of each pair transition
alpha = 1.5

with arcpy.da.SearchCursor("City_FromToTable_Final","*") as cursor:
    total_sum = sum(((row[5] * row[7])/(row[3]**alpha)) for row in cursor)
with arcpy.da.UpdateCursor("City_FromToTable_Final","*") as cursor:
    for row in cursor:
        row[10] = (row[5] * row[7])/(row[3]**alpha)/total_sum
        cursor.updateRow(row)
del cursor

In [96]:
#run 30 simulations
for i in range(0,30):
    PresenceCity = sim_pred("City_FromToTable_Final","City_Pred_0",method = 'G',
                              PresenceCity = ['Minneapolis'],num_month = num_month,G = 15,i = i)
    
    PresenceCity = sim_pred("City_FromToTable_Final","City_Pred_1",method = 'G',PresenceCity = PresenceCity,num_month = 1,G = 15,i = i)
    PresenceCity = sim_pred("City_FromToTable_Final","City_Pred_2",method = 'G',PresenceCity = PresenceCity, num_month = 1, G = 15,i = i)
    PresenceCity = sim_pred("City_FromToTable_Final","City_Pred_3",method = 'G',PresenceCity = PresenceCity, num_month = 1, G = 15,i = i)

Simulation0:['Minneapolis', 'Richfield', 'Maplewood', 'Inver Grove Heights', 'Edina', 'Saint Louis Park', 'Mahtomedi', 'Saint Paul', 'Mendota Heights', 'Shoreview', 'Chanhassen', 'Monticello', 'Plymouth', 'Osseo', 'Bloomington', 'Crystal', 'Lilydale', 'Columbia Heights', 'Brooklyn Center', 'Rosemount', 'Minnetonka', 'Winona', 'Woodbury', 'Brooklyn Park', 'Roseville', 'Lakeville', 'Farmington', 'Little Canada', 'Golden Valley', 'Corcoran', 'Eden Prairie', 'New Brighton', 'Andover', 'Cambridge', 'Hutchinson', 'Hopkins', 'Saint Peter', 'Deephaven', 'Fridley', 'Coon Rapids', 'Arden Hills', 'West Saint Paul', 'Browerville', 'New Hope', 'Maple Grove', 'Cottage Grove']
Simulation1:['Minneapolis', 'Roseville', 'Saint Paul', 'Shoreview', 'Brooklyn Park', 'Lakeville', 'Mahtomedi', 'Coon Rapids', 'Inver Grove Heights', 'West Saint Paul', 'Blaine', 'Mendota Heights', 'New Hope', 'Chaska', 'Mounds View', 'Lino Lakes', 'Robbinsdale', 'South Saint Paul', 'White Bear Lake', 'New Brighton', 'Eagan', 'O

Simulation28:['Minneapolis', 'Columbia Heights', 'Bloomington', 'Cottage Grove', 'Saint Paul', 'Orono', 'Saint Michael', 'Waseca', 'Fridley', 'Roseville', 'Brooklyn Center', 'Burnsville', 'Rogers', 'Spring Lake Park', 'Brooklyn Park', 'Hanley Falls', 'Saint Louis Park', 'New Brighton', 'Mendota Heights', 'Hastings', 'Chaska', 'West Saint Paul', 'Saint Anthony', 'Big Lake', 'Eden Prairie', 'Long Lake', 'Maplewood', 'Minnetonka', 'Apple Valley', 'North Saint Paul', 'Inver Grove Heights', 'Eagan', 'Owatonna', 'Plymouth']
Simulation29:['Minneapolis', 'Edina', 'Arden Hills', 'Mendota Heights', 'Eagan', 'Bloomington', 'Coon Rapids', 'New Ulm', 'Champlin', 'Plymouth', 'Shakopee', 'Spring Lake Park', 'Columbia Heights', 'Anoka', 'Circle Pines', 'Maple Grove', 'Lakeville', 'Chanhassen', 'Rochester', 'Apple Valley', 'East Bethel', 'Brooklyn Park', 'Oakdale', 'Brooklyn Center', 'Golden Valley', 'Falcon Heights', 'Inver Grove Heights', 'Roseville', 'Saint Louis Park', 'Rockford']


In [97]:
#get the simulation final prediction
arcpy.management.AddField("City_Pred_0", f"Pred_G", "SHORT")
arcpy.management.AddField("City_Pred_1", f"Pred_G", "SHORT")
arcpy.management.AddField("City_Pred_2", f"Pred_G", "SHORT")
arcpy.management.AddField("City_Pred_3", f"Pred_G", "SHORT")

final_result("City_Pred_0",method = 'G')        
#delete unnecessary columns
arcpy.management.DeleteField("City_Pred_0", "CITY_NAME;POPULATION;Pred_MC;Pred_G", "KEEP_FIELDS")

final_result("City_Pred_1",method = 'G')
#delete unnecessary columns
arcpy.management.DeleteField("City_Pred_1", "CITY_NAME;POPULATION;Pred_MC;Pred_G", "KEEP_FIELDS")

final_result("City_Pred_2",method = 'G')
#delete unnecessary columns
arcpy.management.DeleteField("City_Pred_2", "CITY_NAME;POPULATION;Pred_MC;Pred_G", "KEEP_FIELDS")

final_result("City_Pred_3",method = 'G')
#delete unnecessary columns
arcpy.management.DeleteField("City_Pred_3", "CITY_NAME;POPULATION;Pred_MC;Pred_G", "KEEP_FIELDS")

<Result 'City_Pred_3'>

### Huff Model - LandUse, Pop, Dist

In [43]:
# adds probability column to near table
arcpy.management.AddField("City_FromToTable_Final", "Probability_H", "DOUBLE")

#Calculates probability of each pair transition
alpha = 1.5

with arcpy.da.SearchCursor("City_FromToTable_Final","*") as cursor:
    total_sum = sum(((row[7] * (row[9]+0.001))/(row[3]**alpha)) for row in cursor)
with arcpy.da.UpdateCursor("City_FromToTable_Final","*") as cursor:
    for row in cursor:
        row[11] = ((row[7] * (row[9]+0.001))/(row[3]**alpha))/total_sum
        cursor.updateRow(row)
del cursor

In [115]:
#run 50 simulations
for i in range(0,30):
    
    PresenceCity = sim_pred("City_FromToTable_Final","City_Pred_0",method = 'H',
                              PresenceCity = ['Minneapolis'],num_month = num_month,G = 110,i = i)
    
    PresenceCity = sim_pred("City_FromToTable_Final","City_Pred_1",method = 'H',PresenceCity = PresenceCity, num_month = 1, G = 110,i = i)
    PresenceCity = sim_pred("City_FromToTable_Final","City_Pred_2",method = 'H',PresenceCity = PresenceCity, num_month = 1, G = 110,i = i)
    PresenceCity = sim_pred("City_FromToTable_Final","City_Pred_3",method = 'H',PresenceCity = PresenceCity, num_month = 1, G = 110,i = i)

Simulation0:['Minneapolis', 'Bloomington', 'Victoria', 'Minnetonka', 'Buffalo', 'New Prague', 'Eyota', 'Shakopee', 'Woodbury', 'Lakeville', 'Mahtomedi', 'Becker', 'Saint Michael', 'Eden Prairie', 'Burnsville', 'Princeton', 'Saint Paul', 'Saint Joseph', 'Byron', 'Watertown', 'Farmington', 'Richfield', 'Columbus', 'Faribault', 'Saint Charles', 'Mounds View', 'Savage', 'Sartell', 'Chaska', 'Edina', 'Plymouth', 'Blue Earth', 'Apple Valley', 'Forest Lake', 'Blaine', 'Maple Grove', 'Hugo', 'Falcon Heights', 'Shorewood', 'Otsego', 'Waseca', 'Sauk Rapids', 'New Brighton', 'Elk River', 'Coon Rapids', 'Roseville', 'Inver Grove Heights']
Simulation1:['Minneapolis', 'West Saint Paul', 'Mendota Heights', 'Roseville', 'Edina', 'Woodbury', 'Saint Paul', 'Waconia', 'Vadnais Heights', 'Faribault', 'Golden Valley', 'Maple Grove', 'Eden Prairie', 'Blaine', 'Rogers', 'Big Lake', 'Arden Hills', 'Ramsey', 'Burnsville', 'Goodview', 'Winona', 'Chaska', 'Inver Grove Heights', 'Ham Lake', 'Shakopee', 'Belle Pla

Simulation25:['Minneapolis', 'Maple Grove', 'Rogers', 'Saint Michael', 'Lino Lakes', 'Corcoran', 'Mound', 'Farmington', 'Rosemount', 'Shakopee', 'Hugo', 'Ramsey', 'Saint Francis', 'Prior Lake', 'Apple Valley', 'Aitkin', 'Montgomery', 'Burnsville', 'Inver Grove Heights', 'Bloomington', 'Stillwater', 'Columbus', 'Delano', 'Brooklyn Park', 'Saint Louis Park', 'Victoria', 'Eden Prairie', 'Cottage Grove', 'Lakeville', 'Vadnais Heights', 'Coon Rapids', 'Wyoming', 'Mendota Heights', 'Dayton', 'Coates', 'Andover', 'Greenfield', 'New Prague', 'Hanover', 'North Branch', 'Eagan', 'Saint Paul', 'Chaska', 'Forest Lake', 'Blaine', 'Shorewood', 'Saint Cloud', 'Barnesville', 'Rochester', 'Mankato', 'Orono', 'Benson']
Simulation26:['Minneapolis', 'Brooklyn Park', 'Plymouth', 'Lino Lakes', 'Brooten', 'Coon Rapids', 'Oak Park Heights', 'Crystal', 'Saint Paul', 'Maple Grove', 'Eagan', 'Fridley', 'Blaine', 'Shoreview', 'Andover', 'Shakopee', 'Robbinsdale', 'Rosemount', 'Hastings', 'Credit River', 'Lakevill

In [116]:
#get the simulation final prediction
arcpy.management.AddField("City_Pred_0", f"Pred_H", "SHORT")
arcpy.management.AddField("City_Pred_1", f"Pred_H", "SHORT")
arcpy.management.AddField("City_Pred_2", f"Pred_H", "SHORT")
arcpy.management.AddField("City_Pred_3", f"Pred_H", "SHORT")

final_result("City_Pred_0",method = 'H')        
#delete unnecessary columns
arcpy.management.DeleteField("City_Pred_0", "CITY_NAME;POPULATION;Pred_MC;Pred_G;Pred_H", "KEEP_FIELDS")

final_result("City_Pred_1",method = 'H')
#delete unnecessary columns
arcpy.management.DeleteField("City_Pred_1", "CITY_NAME;POPULATION;Pred_MC;Pred_G;Pred_H", "KEEP_FIELDS")

final_result("City_Pred_2",method = 'H')
#delete unnecessary columns
arcpy.management.DeleteField("City_Pred_2", "CITY_NAME;POPULATION;Pred_MC;Pred_G;Pred_H", "KEEP_FIELDS")

final_result("City_Pred_3",method = 'H')
#delete unnecessary columns
arcpy.management.DeleteField("City_Pred_3", "CITY_NAME;POPULATION;Pred_MC;Pred_G;Pred_H", "KEEP_FIELDS")

<Result 'City_Pred_3'>

### Assessment comparision for three models

In [100]:
#Spatial Join BMSB with CITYNAME AND POPULATION for further assessment
arcpy.SpatialJoin_analysis('bmsb', "City_qaqc_done", 'bmsb_SpatialJoin',join_operation="JOIN_ONE_TO_ONE", match_option = "CLOSEST_GEODESIC")

# get true presence cities
bmsb_city = []
with arcpy.da.SearchCursor("bmsb_SpatialJoin", ["CITY_NAME"]) as cursor:
    for row in cursor:
        if row[0] is not None:
            bmsb_city.append(row[0])
            
bmsb_city = list(set(bmsb_city))

In [101]:
bmsb_city

['Brooklyn Park', 'Minneapolis', 'Prior Lake', 'Saint Cloud', 'Victoria', 'Edina', 'Wyoming', 'Grant', 'North Mankato', 'Maplewood', 'Rosemount', 'Roseville', 'Cannon Falls', 'Afton', 'Carver', 'Woodbury', 'Cottage Grove', 'Apple Valley', 'Empire', 'Farmington', 'Jordan', 'Shoreview', 'Blaine', 'Hastings', 'Saint Paul', 'Northfield']

In [117]:
#confusion matrix
def get_ConfusionMatrix(pred,name,truth,TP,TN,FP,FN):
    if (pred == 1) and (name in truth):
        TP +=1
    elif (pred == 0) and (name not in truth):
        TN +=1
    elif (pred == 1) and (name not in truth):
        FP +=1
    elif (pred == 0) and (name in truth):
        FN +=1
    return TP,TN,FP,FN

TP_G,TN_G,FP_G,FN_G = 0,0,0,0
TP_MC,TN_MC,FP_MC,FN_MC = 0,0,0,0
TP_H,TN_H,FP_H,FN_H = 0,0,0,0

with arcpy.da.SearchCursor("City_Pred_0", ["CITY_NAME","Pred_G",'Pred_MC','Pred_H']) as cursor:
    for row in cursor:
        TP_G,TN_G,FP_G,FN_G = get_ConfusionMatrix(row[1],row[0],bmsb_city,TP_G,TN_G,FP_G,FN_G)
        TP_MC,TN_MC,FP_MC,FN_MC = get_ConfusionMatrix(row[2],row[0],bmsb_city,TP_MC,TN_MC,FP_MC,FN_MC)
        TP_H,TN_H,FP_H,FN_H = get_ConfusionMatrix(row[3],row[0],bmsb_city,TP_H,TN_H,FP_H,FN_H)

In [118]:
#accuracy
accuracy_MC =  (TP_MC + TN_MC) / (TP_MC + TN_MC + FP_MC + FN_MC)
accuracy_G =  (TP_G + TN_G) / (TP_G + TN_G + FP_G + FN_G)
accuracy_H =  (TP_H + TN_H) / (TP_H + TN_H + FP_H + FN_H)
print(f"The accuracy for monte carlo model is:{accuracy_MC}")
print(f"The accuracy for gravity model is:{accuracy_G}")
print(f"The accuracy for huff model is:{accuracy_H}")

#precision
precision_MC = TP_MC / (TP_MC + FP_MC)
precision_G = TP_G / (TP_G + FP_G)
precision_H = TP_H / (TP_H + FP_H)
print(f"The precision for monte carlo model is:{precision_MC}")
print(f"The precision for gravity model is:{precision_G}")
print(f"The precision for huff model is:{precision_H}")

#recall
recall_MC = TP_MC / (TP_MC + FN_MC)
recall_G = TP_G / (TP_G + FN_G)
recall_H = TP_H / (TP_H + FN_H)
print(f"The recall for monte carlo model is:{recall_MC}")
print(f"The recall for gravity model is:{recall_G}")
print(f"The recall for huff model is:{recall_H}")

The accuracy for monte carlo model is:0.9437939110070258
The accuracy for gravity model is:0.9566744730679156
The accuracy for huff model is:0.9590163934426229
The precision for monte carlo model is:0.15625
The precision for gravity model is:0.3225806451612903
The precision for huff model is:0.3448275862068966
The recall for monte carlo model is:0.19230769230769232
The recall for gravity model is:0.38461538461538464
The recall for huff model is:0.38461538461538464


### Get the priority monitoring city

In [122]:
#first priority cities (FP in 2021)
# get true presence cities
city2021 = []
with arcpy.da.SearchCursor("City_Pred_0", ["CITY_NAME",'Pred_H']) as cursor:
    for row in cursor:
        if row[1] == 1 and row[0] not in bmsb_city:
            city2021.append(row[0])

#second priority cities (will spread in 2022)
city2022 = []
with arcpy.da.SearchCursor("City_Pred_1", ["CITY_NAME",'Pred_H']) as cursor:
    for row in cursor:
        if row[1] == 1 and row[0] not in bmsb_city and row[0] not in city2021:
            city2022.append(row[0])

#third priority cities (will spread in 2023)
city2023 = []
with arcpy.da.SearchCursor("City_Pred_2", ["CITY_NAME",'Pred_H']) as cursor:
    for row in cursor:
        if row[1] == 1 and row[0] not in bmsb_city and row[0] not in city2021 and row[0] not in city2022:
            city2023.append(row[0])
            
#forth priority cities (will spread in 2024)
city2024 = []
with arcpy.da.SearchCursor("City_Pred_3", ["CITY_NAME",'Pred_H']) as cursor:
    for row in cursor:
        if row[1] == 1 and row[0] not in bmsb_city and row[0] not in city2021 and row[0] not in city2022 and row[0] not in city2023:
            city2024.append(row[0])

In [123]:
arcpy.management.AddField("City_qaqc_done", f"Priority", "SHORT")
with arcpy.da.UpdateCursor("City_qaqc_done", ["CITY_NAME",'Priority']) as cursor:
    for row in cursor:
        if row[0] in city2021:
            row[1] = 5
        elif row[0] in city2022:
            row[1] = 4
        elif row[0] in city2023:
            row[1] = 3
        elif row[0] in city2024:
            row[1] = 2
        else:
            row[1] = 1
        cursor.updateRow(row)

In [119]:
TP_G,TN_G,FP_G,FN_G

(10, 807, 21, 16)

In [120]:
TP_MC,TN_MC,FP_MC,FN_MC

(5, 801, 27, 21)

In [121]:
TP_H,TN_H,FP_H,FN_H

(10, 809, 19, 16)